In [1]:
# import required modules
%cd /home/gridsan/lchan/git-remotes/polychrom_analysis
import os
import sys
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import matplotlib.colors as colors
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable

import polychrom
from polychrom.hdf5_format import list_URIs, load_URI, load_hdf5_file

from post_processing.visualization import *
from post_processing.analysis import *
from post_processing.compscores import *
from post_processing.msd import *

import random
import csv
import math
from scipy.stats import rv_continuous

import nglutils as ngu
import nglview as nv

/home/gridsan/lchan/git-remotes/polychrom_analysis


In [ ]:
# for visualization of polymer
%load_ext autoreload
%autoreload 2

In [2]:
# load chromosomal data
ABids = np.loadtxt("data/ABidentities_chr21_Su2020_2perlocus.csv", dtype=str) # make sure to change this!
ids = (ABids == "A").astype(int)

# parameter sweep values
param_set = []
act_ratio = [1, 2, 3, 4, 5]
e0 = [0, 0.075, 0.15, 0.225, 0.3]
for a in act_ratio:
    for e in e0:
        param_set.append((a, e))

simnames = [f'stickyBB_{BBenergy}_act{act_ratio}' for (act_ratio, BBenergy) in param_set]

In [4]:
# overwrite method for parameter defaults
start_i = 1
end_i = 2
def extract(path, start=start_i, every_other=1, end=end_i): # put here to modify start/ends. run before previous
    """Extract independent snapshots from a single simulation.

    Parameters
    ----------
    path : str
        path to simulation directory
    start : int
        block number to start extracting files from
    every_other : int
        skip this number of blocks in between snapshots (should be large enough
        so that snapshots are decorrelated)
    end : int
        last block to include in trajectory.
    """
    try:
        confs = list_URIs(path)
        if end:
            uris = confs[start:end:every_other]
        else:
            uris = confs[start::every_other]
    except:
        raise Exception("Exception! Something went wrong")
        uris = []
    return uris
def extract_conformations(basepath, ncores=24, chain=True, **kwargs):
    """Extract conformations from multiple simulation replicates to be included in
    ensemble-averaged observables.

    Parameters
    ----------
    basepath : str or Path
        parent directory where each subdirectory is a simulation replicate for one set of parameters
    ncores : int
        number of cores available for parallelization
    chain : bool
        whether to aggregate conformations from multiple simulations into one list.
        Defaults to True.

    Returns
    -------
    conformations : list
        If chain is True, list of hdf5 filenames containing polymer conformations.
        If chain is False, list of lists, where each sublist is from a separate simulation run.
    """
    basepath = Path(basepath)
    rundirs = [f for f in basepath.iterdir() if f.is_dir()]
    runs = len(rundirs)
    extract_func = partial(extract, **kwargs)
    with mp.Pool(ncores) as p:
        confs = p.map(extract_func, rundirs)
    if chain:
        conformations = list(itertools.chain.from_iterable(confs))
        print(f"Number of simulations in directory: {runs}")
        print(f"Number of conformations extracted: {len(conformations)}")
        return conformations, runs
    else:
        return confs, runs
def extract_hot_cold(simdir, D, start=100000, every_other=10):
    """Load conformations from a simulation trajectory stored in the hdf5 files in simdir
    and store in two matrices, one for the `A` type monomers, and one for the `B` monomers.

    Parameters
    ----------
    simdir : str or Path
        path to simulation directory containing .h5 files
    D : np.ndarray
        array of monomer diffusion coefficients.
        Assumes there are only 2 values: D.min() and D.max().
    start : int
        which time block to start loading conformations from
    every_other : int
        skip every_other time steps when loading conformations

    Returns
    -------
    Xhot : array_like (num_t, N_A, 3)
        x, y, z positions of all N_A active (hot) monomers over time
    Xcold : array-like (num_t, N_B, 3)
        x, y, z positions of all N_B inactive (cold) monomers over time

    """
    X = []
    N = len(D)
    data = list_URIs(simdir)
    if start == 0:
        starting_pos = load_hdf5_file(Path(simdir) / "starting_conformation_0.h5")[
            "pos"
        ]
        X.append(starting_pos)
    for conformation in data[start::every_other]:
        pos = load_URI(conformation)["pos"]
        ncopies = pos.shape[0] // N
        for i in range(ncopies):
            posN = pos[N * i : N * (i + 1)]
            X.append(posN)
    X = np.array(X)
    print(X.shape)
    Xcold = X[:, D == D.min(), :]
    Xhot = X[:, D == D.max(), :]
    return Xhot, Xcold


In [23]:
# extract some conformations
chromo = 'normal_logclustered'
simname = simnames[0]
path = f'/home/gridsan/lchan/git-remotes/polychrom_analysis/artificial_chr/{chromo}/{simname}'
start_i = 0
end_i = 1109
conformations, runs = extract_conformations(path)
np.save(f'/home/gridsan/lchan/git-remotes/polychrom_analysis/data/conformations/{chromo}/{simname}', conformations)

Number of simulations in directory: 1
Number of conformations extracted: 1099


In [7]:
# function to calculate MSDs and store as csv in data directory
def save_MSD_time_ave(simpath, ids, savepath, every_other=11):
    """Compute time lag averaged MSDs averaged over active and inactive regions
    from a single simulation trajectory in simpath. Takes ~30 min for a simulation with
    10,000 conformations.

    Parameters
    ----------
    simpath : str or Path
        path to simulation directory
    ids : array-like
        array where D==D.max() selects out A monomers and D==D.min() selects B monomers
    savepath : str or Path
        path to .csv file where MSDs will be saved
    every_other : int
        skip every_other conformation when loading conformations for MSD computation
    """
    #if not Path(savepath).exists():
    Xhot, Xcold = extract_hot_cold(Path(simpath), ids, start=1, every_other=every_other)
    hot_msd, cold_msd = get_bead_msd_time_ave(Xhot, Xcold)
    df_msd = pd.DataFrame()
    df_msd["Times"] = np.arange(0, len(hot_msd)) * every_other
    df_msd["MSD_A"] = hot_msd
    df_msd["MSD_B"] = cold_msd
    df_msd.to_csv(savepath)

In [ ]:
# save MSDs to file. make sure to change directory to proper chromosome!
simnames = ['stickyBB_0_act1'] # change this when done debug
for simname in simnames:
    save_MSD_time_ave(f'/home/gridsan/lchan/git-remotes/polychrom_analysis/artificial_chr/normal_logclustered/{simname}/runs1100_0_200copies', ids, 
                      f'/home/gridsan/lchan/git-remotes/polychrom_analysis/data/msds/normal_logclustered/{simname}.csv')

In [ ]:
# functions to plot MSDs
def plot_msds(stickiness, act, title_append='', simname=None, ax=None):
    if simname:
        df = pd.read_csv(datapath/f'{simname}.csv')
    else:
        print('gg')
    if ax is None:
        fig, ax = plt.subplots()
    ax.loglog(df['Times'].values[0::20], df['MSD_A'].values[0::20], 'r-', label='A')
    ax.loglog(df['Times'].values[0::20], df['MSD_B'].values[0::20], 'b-', label='B')
    ax.set_xlabel('time blocks')
    ax.set_ylabel('MSD')
    ax.legend()
    ax.set_title(f'$E_BB$ = {stickiness}, $A_A / A_B$ = {act}{title_append}')
    
def plot_msd_panel(simpaths):
    num_rows = 5
    num_cols = 5
    fig = plt.figure(figsize=(3*num_cols, 2 + 2*num_rows))
    #extract activity / stickiness from simpath name
    BBenergy = np.array([float(simname.split('_')[1]) for simname in simpaths])
    activities = np.array([float(simname.split('act')[1]) for simname in simpaths])
    if np.all(activities == activities[0]):
        fig.suptitle(f'Mean squared displacements for activity {activities[0]}', 
                     fontsize=14, fontweight='bold')
    elif np.all(BBenergy == BBenergy[0]):
        fig.suptitle(f'Mean squared displacements for stickiness {BBenergy[0]}', 
                     fontsize=14, fontweight='bold')
    else:
        fig.suptitle('Mean squared displacements', 
                    fontsize=14, fontweight='bold')
        
    gs = GridSpec(nrows=num_rows, ncols=num_cols, 
                  width_ratios=[100]*num_cols)
    
    for i in range(len(simpaths)):
        x = i // num_cols
        y = i % num_cols
        ax = fig.add_subplot(gs[x, y])
        plot_msds(BBenergy[i], activities[i], simname=simpaths[i], ax=ax)
        if np.all(activities == activities[0]):
            ax.set_title(r'$E_{BB}=$' + f'${BBenergy[i]}kT$', fontsize=12)
        elif np.all(BBenergy == BBenergy[0]):
            ax.set_title(f'$A_A/A_B={activities[i]}$', fontsize=12)
        else:
            ax.set_title(r'$E_{BB}=$' + f'${BBenergy[i]}kT$' + f' $A_A/A_B={activities[i]}$', fontsize=12)
    fig.tight_layout()
    plt.show()

In [ ]:
# plot MSDs
datapath = Path('/home/gridsan/lchan/git-remotes/polychrom_analysis/data/msds/normal_logclustered')
plot_msds(0, 1, '', 'stickyBB_0_act1')
simpaths = [f'stickyBB_{BBenergy}_act{act_ratio}' for (act_ratio, BBenergy) in param_set]
#plot_msd_panel(simpaths)